In [6]:
import os
import glob
import math
import time
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import statsmodels.api as sm
import pylab as pl
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import umap
from sklearn import svm
from lightgbm import LGBMClassifier
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from k_means_constrained import KMeansConstrained

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [9]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')

## 1.3 Show the maximum return of train and test

In [10]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


### 1.3.1 Remove the Unnamed columns in dataframe

In [11]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

In [12]:
train

0            1            2            3          4  \
0       5874.524387  1072.671848  41440.76212  41440.23732  70.405148   
1       6124.154099  1072.802927  41440.76212  41442.22458  70.456758   
2       5905.732593  1072.802927  41443.14358  41442.30403  70.422472   
3       6029.325221  1072.802927  41440.76212  41442.25682  70.458533   
4       6142.360146  1072.802927  41443.14358  41442.46480  70.413623   
...             ...          ...          ...          ...        ...   
89995  13562.064960  1061.530132  41235.95694  41236.96634  70.431821   
89996  12711.732070  1061.530132  41238.33840  41236.82293  70.459676   
89997  11857.854050  1061.530132  41235.95694  41236.73441  70.419055   
89998  11803.230780  1061.530132  41238.33840  41236.77811  70.462143   
89999  13473.515930  1061.530132  41235.95694  41237.03341  70.435513   

              5          6            7            8          9        10  \
0      7.392780  70.377281  23229.69262  23229.72655  70.378864  7.389173   
1      7.356050  70.379576  23229.76020  23230.10472  70.273618  7.389813   
2      7.417794  70.376448  23229.48142  23229.61008  70.474265  7.388979   
3      7.366031  70.379221  23230.08433  23229.87971  70.288944  7.390120   
4      7.411287  70.376788  23229.70975  23229.82255  70.467206  7.389910   
...         ...        ...          ...          ...        ...       ...   
89995  7.414020  70.378048  23229.76548  23229.77954  70.373676  7.389395   
89996  7.363988  70.376757  23229.77584  23229.79672  70.371404  7.389131   
89997  7.413537  70.377374  23229.81430  23229.80621  70.385575  7.389776   
89998  7.367474  70.376032  23229.79226  23229.77640  70.369925  7.388977   
89999  7.404631  70.378495  23229.68359  23229.78133  70.368750  7.389127   

              11           12           13         14         15         16  \
0      70.380160  23229.76782  23379.81637  83.418623  11.615135  83.418623   
1      70.329906  23229.46908  23384.98219  83.418623  11.615135  83.418623   
2      70.397301  23229.83396  23387.39168  83.418623  11.615135  83.418623   
3      70.370247  23229.81662  23390.02296  83.418623  11.615135  83.418623   
4      70.386986  23229.87603  23392.66710  70.572881  11.615135  83.418623   
...          ...          ...          ...        ...        ...        ...   
89995  70.377297  23229.78250  25587.29752  70.703960   7.495239  70.703960   
89996  70.372640  23229.74768  25589.76937  70.835039   7.537712  70.835039   
89997  70.382216  23229.84369  25594.86132  70.703960   7.495239  70.703960   
89998  70.374728  23229.76390  25597.41959  70.835039   7.537712  70.835039   
89999  70.368700  23229.68830  25599.96799  70.703960   7.495239  70.703960   

                17           18         19        20         21           22  \
0      23466.72590  23466.72590  83.418623  7.425618  70.494241  23229.77107   
1      23466.72590  23466.72590  83.418623  7.441223  70.702624  23229.77107   
2      23466.72590  23466.72590  83.418623  7.440201  70.686178  23229.77107   
3      23233.34325  23466.72590  83.418623  7.439191  70.692085  23229.77107   
4      23466.72590  23466.72590  83.418623  7.438244  70.680386  23229.77107   
...            ...          ...        ...       ...        ...          ...   
89995  23238.10616  23238.10616  70.835039  7.468157  70.432175  23230.96180   
89996  23235.72471  23235.72471  70.703960  7.468103  70.438991  23230.96180   
89997  23238.10616  23238.10616  70.835039  7.468050  70.423227  23230.96180   
89998  23235.72471  23235.72471  70.703960  7.467892  70.421787  23230.96180   
89999  23238.10616  23238.10616  70.835039  7.467868  70.433799  23230.96180   

                23         24        25         26           27           28  \
0      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
1      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
2      23229.77107  70.376262  7.389056  70.376262  23229.77107  2

In [13]:
# train_ = (train.iloc[:, :-1]-train.iloc[:, :-1].mean())/train.iloc[:, :-1].std()
# train_['TARGET'] = train['TARGET']
# train = train_

# test_ = (test.iloc[:, :-1]-test.iloc[:, :-1].mean())/test.iloc[:, :-1].std()
# test_['TARGET'] = test['TARGET']
# test = test_

In [14]:
# train['group'] = 1

# quantiles = [train['TARGET'].quantile(x/10) for x in range(1, 10)]

# for i in range(train.shape[0]):
#     for j in range(9):
#         if train['TARGET'][i] < quantiles[j]:
#             train['group'][i] = j
#             break
#         train['group'][i] = 9

In [15]:
# test['group'] = 1

# # quantiles = [test['TARGET'].quantile(x/10) for x in range(1, 10)]

# for i in range(test.shape[0]):
#     for j in range(9):
#         if test['TARGET'][i] < quantiles[j]:
#             test['group'][i] = j
#             break
#         test['group'][i] = 9

In [16]:
# fig, ax = plt.subplots(10, 3, figsize = (15, 20))

# quantiles_num = []
# train_sum_returns = []
# test_sum_returns = []

# models = []

# for i in range(10):
#     train_seg = train[train['group']==i]
#     test_seg = test[test['group']==i]
    
#     quantiles_num.append([train_seg, test_seg])
    
#     reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
#     models.append(reg)
    
#     pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

#     pred_test = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

#     train_res = np.sum(train_seg['TARGET'][pred>0])
#     test_res = np.sum(test_seg['TARGET'][pred_test>0])
     
#     train_sum_returns.append(train_res)
#     test_sum_returns.append(test_res)
    
#     max_value = max(max(train_seg['TARGET']), max(pred))
#     min_value = min(min(train_seg['TARGET']), min(pred))
#     ax[i, 0].scatter(train_seg['TARGET'], pred, c = 'blue')
#     ax[i, 0].plot([min_value, max_value], [min_value, max_value], 'k-')
#     ax[i, 0].set_xlim([min_value, max_value])
#     ax[i, 0].set_ylim([min_value, max_value])
#     ax[i, 0].set_xlabel('train dataset target')
#     ax[i, 0].set_ylabel('predicted target')
# #     ax[i, 0].set_title('Sactter plot of train target vs prediction')
    
#     max_value = max(max(test_seg['TARGET']), max(pred_test))
#     min_value = min(min(test_seg['TARGET']), min(pred_test))
#     ax[i, 1].scatter(test_seg['TARGET'], pred_test, c = 'blue')
#     ax[i, 1].plot([min_value, max_value], [min_value, max_value], 'k-')
#     ax[i, 1].set_xlim([min_value, max_value])
#     ax[i, 1].set_ylim([min_value, max_value])
#     ax[i, 1].set_xlabel('test dataset target')
#     ax[i, 1].set_ylabel('predicted target')
# #     ax[i, 1].set_title('Sactter plot of test target vs prediction')
    
#     ax[i, 2].scatter(train_seg['TARGET'], train_seg['TARGET'], alpha=0.2, c = 'blue')
# #     ax[i, 2].set_title('Train target points scatter plot')
#     ax[i, 2].set_xlabel('Target')
#     ax[i, 2].set_ylabel('Target')

#     fig.tight_layout()
# #     plt.show()

In [17]:
# print(sum(train_sum_returns))
# print(sum(test_sum_returns))

In [18]:
# train_np = train.iloc[:, :66].to_numpy()
# test_np = test.iloc[:, :66].to_numpy()

In [19]:
# kmeans = KMeans(n_clusters=100, random_state=0).fit(train_np)
# train_pred = kmeans.predict(train_np)
# test_pred = kmeans.predict(test_np)
# train['pred_class'] = train_pred
# test['pred_class'] = test_pred

In [20]:
# fig, ax = plt.subplots(100, 4, figsize = (20, 200))

# quantiles_num = []
# train_sum_returns = []
# test_sum_returns = []

# models = []

# for i in range(100):
    
#     train_seg = train[train['pred_class']==i]
#     test_seg = test[test['pred_class']==i]
#     reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
#     models.append(reg)
    
#     pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

#     pred_test = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

#     train_res = np.sum(train_seg['TARGET'][pred>0])
#     test_res = np.sum(test_seg['TARGET'][pred_test>0])
    
#     train_sum_returns.append(train_res)
#     test_sum_returns.append(test_res)
    
#     max_value = max(max(train_seg['TARGET']), max(pred))
#     min_value = min(min(train_seg['TARGET']), min(pred))
#     ax[i, 0].scatter(train_seg['TARGET'], pred, c = 'blue')
#     ax[i, 0].plot([min_value, max_value], [min_value, max_value], 'k-')
#     ax[i, 0].set_xlim([min_value, max_value])
#     ax[i, 0].set_ylim([min_value, max_value])
#     ax[i, 0].set_xlabel('train dataset target')
#     ax[i, 0].set_ylabel('predicted target')
# #     ax[i, 0].set_title('Sactter plot of train target vs prediction')
    
#     max_value = max(max(test_seg['TARGET']), max(pred_test))
#     min_value = min(min(test_seg['TARGET']), min(pred_test))
#     ax[i, 1].scatter(test_seg['TARGET'], pred_test, c = 'blue')
#     ax[i, 1].plot([min_value, max_value], [min_value, max_value], 'k-')
#     ax[i, 1].set_xlim([min_value, max_value])
#     ax[i, 1].set_ylim([min_value, max_value])
#     ax[i, 1].set_xlabel('test dataset target')
#     ax[i, 1].set_ylabel('predicted target')
    
#     pca = PCA(n_components=2)
#     pca.fit(train_seg.iloc[:, :66])
#     train_pca = pca.fit_transform(train_seg.iloc[:, :66])
#     reducer = umap.UMAP()
#     train_umap = reducer.fit_transform(train_seg.iloc[:, :66])
    
#     color_map = (train_seg['TARGET'] - np.min(train_seg['TARGET'])) / (np.max(train_seg['TARGET']) - np.min(train_seg['TARGET']))*2-1

#     # ax.scatter(train_pca[:, 0], train_pca[:, 1], c=np.sign(train['TARGET']), cmap=matplotlib.colors.ListedColormap(colors))
#     im_0 = ax[i, 2].scatter(train_pca[:, 0], train_pca[:, 1], c=color_map, cmap='Reds', alpha = 1)
#     im_1 = ax[i, 3].scatter(train_umap[:, 0], train_umap[:, 1], c=color_map, cmap='Reds', alpha = 1)

#     ax[i, 1].set_title('Sactter plot of test target vs prediction')

In [21]:
# print(sum(train_sum_returns))
# print(sum(test_sum_returns))

In [22]:
# train = train.drop(['group', 'pred_class'], axis=1)

In [23]:
# test = test.drop([group', 'pred_class'], axis=1)

In [24]:
train_test = pd.concat([train, test], axis=0)

In [25]:
train_test

0            1            2            3          4  \
0      5874.524387  1072.671848  41440.76212  41440.23732  70.405148   
1      6124.154099  1072.802927  41440.76212  41442.22458  70.456758   
2      5905.732593  1072.802927  41443.14358  41442.30403  70.422472   
3      6029.325221  1072.802927  41440.76212  41442.25682  70.458533   
4      6142.360146  1072.802927  41443.14358  41442.46480  70.413623   
...            ...          ...          ...          ...        ...   
19702   623.185510  1092.989093  41809.88774  41809.29756  70.408747   
19703   566.358102  1092.989093  41807.50629  41809.52337  70.487285   
19704   552.829988  1092.989093  41809.88774  41809.48899  70.398210   
19705   505.454192  1092.989093  41807.50629  41809.58664  70.490768   
19706   329.767251  1092.923553  41809.88774  41807.50629  70.507341   

              5          6            7            8          9        10  \
0      7.392780  70.377281  23229.69262  23229.72655  70.378864  7.389173   
1      7.356050  70.379576  23229.76020  23230.10472  70.273618  7.389813   
2      7.417794  70.376448  23229.48142  23229.61008  70.474265  7.388979   
3      7.366031  70.379221  23230.08433  23229.87971  70.288944  7.390120   
4      7.411287  70.376788  23229.70975  23229.82255  70.467206  7.389910   
...         ...        ...          ...          ...        ...       ...   
19702  7.404133  70.374689  23229.84973  23229.73803  70.107169  7.388319   
19703  7.364694  70.375071  23229.59198  23229.70421  70.611530  7.388002   
19704  7.416755  70.376942  23229.74755  23229.71478  70.158769  7.388783   
19705  7.356461  70.376557  23229.77238  23229.78213  70.569803  7.389092   
19706  7.390135  70.356207  23228.89593  23229.31298  70.089650  7.379225   

              11           12           13         14         15         16  \
0      70.380160  23229.76782  23379.81637  83.418623  11.615135  83.418623   
1      70.329906  23229.46908  23384.98219  83.418623  11.615135  83.418623   
2      70.397301  23229.83396  23387.39168  83.418623  11.615135  83.418623   
3      70.370247  23229.81662  23390.02296  83.418623  11.615135  83.418623   
4      70.386986  23229.87603  23392.66710  70.572881  11.615135  83.418623   
...          ...          ...          ...        ...        ...        ...   
19702  70.370818  23229.72682  25650.85076  70.835039   8.302229  70.835039   
19703  70.369797  23229.65920  25653.29478  71.621513   8.004917  72.276908   
19704  70.376050  23229.74020  25655.86749  70.835039   8.302229  70.835039   
19705  70.374117  23229.75885  25658.35022  71.621513   8.004917  72.276908   
19706  70.302103  23228.79109  25663.46475  70.769499   8.280992  70.769499   

                17           18         19        20         21           22  \
0      23466.72590  23466.72590  83.418623  7.425618  70.494241  23229.77107   
1      23466.72590  23466.72590  83.418623  7.441223  70.702624  23229.77107   
2      23466.72590  23466.72590  83.418623  7.440201  70.686178  23229.77107   
3      23233.34325  23466.72590  83.418623  7.439191  70.692085  23229.77107   
4      23466.72590  23466.72590  83.418623  7.438244  70.680386  23229.77107   
...            ...          ...        ...       ...        ...          ...   
19702  23252.39490  23264.30217  72.276908  7.483477  70.587356  23228.58034   
19703  23238.10616  23280.97236  70.835039  7.483467  70.474878  23233.34325   
19704  23252.39490  23264.30217  72.276908  7.483499  70.475951  23228.58034   
19705  23238.10616  23280.97236  70.835039  7.483143  70.476483  23233.34325   
19706  23253.58562  23265.49290  72.342447  7.485192  70.742529  23229.77107   

                23         24        25         26           27           28  \
0      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
1      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
2      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107 

In [26]:
train_test_np = train_test.iloc[:, :66].to_numpy()

In [22]:
kmeans = KMeans(n_clusters=5000, random_state=0).fit(train_test_np)
train_np = train_test.iloc[:90000, :66].to_numpy()
test_np = train_test.iloc[90000:, :66].to_numpy()

train_pred = kmeans.predict(train_np)
test_pred = kmeans.predict(test_np)
train['pred_class'] = train_pred
test['pred_class'] = test_pred

In [23]:
quantiles_num = []
train_sum_returns = []
test_sum_returns = []

models = []

for i in range(100):
    
    train_seg = train[train['pred_class']==i]
    test_seg = test[test['pred_class']==i]
    if train_seg.shape[0] == 1:
        continue
    if test_seg.shape[0] < 1:
        continue
    reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
    models.append(reg)
    
    pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

    pred_test = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

    train_res = np.sum(train_seg['TARGET'][pred>0])
    test_res = np.sum(test_seg['TARGET'][pred_test>0])
    
    train_sum_returns.append(train_res)
    test_sum_returns.append(test_res)

In [24]:
train_test_cali = pd.concat([train, test, test, test, test, test], axis=0)

In [25]:
train_test_cali

0            1            2            3          4  \
0      5874.524387  1072.671848  41440.76212  41440.23732  70.405148   
1      6124.154099  1072.802927  41440.76212  41442.22458  70.456758   
2      5905.732593  1072.802927  41443.14358  41442.30403  70.422472   
3      6029.325221  1072.802927  41440.76212  41442.25682  70.458533   
4      6142.360146  1072.802927  41443.14358  41442.46480  70.413623   
...            ...          ...          ...          ...        ...   
19702   623.185510  1092.989093  41809.88774  41809.29756  70.408747   
19703   566.358102  1092.989093  41807.50629  41809.52337  70.487285   
19704   552.829988  1092.989093  41809.88774  41809.48899  70.398210   
19705   505.454192  1092.989093  41807.50629  41809.58664  70.490768   
19706   329.767251  1092.923553  41809.88774  41807.50629  70.507341   

              5          6            7            8          9        10  \
0      7.392780  70.377281  23229.69262  23229.72655  70.378864  7.389173   
1      7.356050  70.379576  23229.76020  23230.10472  70.273618  7.389813   
2      7.417794  70.376448  23229.48142  23229.61008  70.474265  7.388979   
3      7.366031  70.379221  23230.08433  23229.87971  70.288944  7.390120   
4      7.411287  70.376788  23229.70975  23229.82255  70.467206  7.389910   
...         ...        ...          ...          ...        ...       ...   
19702  7.404133  70.374689  23229.84973  23229.73803  70.107169  7.388319   
19703  7.364694  70.375071  23229.59198  23229.70421  70.611530  7.388002   
19704  7.416755  70.376942  23229.74755  23229.71478  70.158769  7.388783   
19705  7.356461  70.376557  23229.77238  23229.78213  70.569803  7.389092   
19706  7.390135  70.356207  23228.89593  23229.31298  70.089650  7.379225   

              11           12           13         14         15         16  \
0      70.380160  23229.76782  23379.81637  83.418623  11.615135  83.418623   
1      70.329906  23229.46908  23384.98219  83.418623  11.615135  83.418623   
2      70.397301  23229.83396  23387.39168  83.418623  11.615135  83.418623   
3      70.370247  23229.81662  23390.02296  83.418623  11.615135  83.418623   
4      70.386986  23229.87603  23392.66710  70.572881  11.615135  83.418623   
...          ...          ...          ...        ...        ...        ...   
19702  70.370818  23229.72682  25650.85076  70.835039   8.302229  70.835039   
19703  70.369797  23229.65920  25653.29478  71.621513   8.004917  72.276908   
19704  70.376050  23229.74020  25655.86749  70.835039   8.302229  70.835039   
19705  70.374117  23229.75885  25658.35022  71.621513   8.004917  72.276908   
19706  70.302103  23228.79109  25663.46475  70.769499   8.280992  70.769499   

                17           18         19        20         21           22  \
0      23466.72590  23466.72590  83.418623  7.425618  70.494241  23229.77107   
1      23466.72590  23466.72590  83.418623  7.441223  70.702624  23229.77107   
2      23466.72590  23466.72590  83.418623  7.440201  70.686178  23229.77107   
3      23233.34325  23466.72590  83.418623  7.439191  70.692085  23229.77107   
4      23466.72590  23466.72590  83.418623  7.438244  70.680386  23229.77107   
...            ...          ...        ...       ...        ...          ...   
19702  23252.39490  23264.30217  72.276908  7.483477  70.587356  23228.58034   
19703  23238.10616  23280.97236  70.835039  7.483467  70.474878  23233.34325   
19704  23252.39490  23264.30217  72.276908  7.483499  70.475951  23228.58034   
19705  23238.10616  23280.97236  70.835039  7.483143  70.476483  23233.34325   
19706  23253.58562  23265.49290  72.342447  7.485192  70.742529  23229.77107   

                23         24        25         26           27           28  \
0      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
1      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107   
2      23229.77107  70.376262  7.389056  70.376262  23229.77107  23229.77107 

In [1]:
train_test_cali_np = train_test_cali.iloc[:, :66].to_numpy()

NameError: name 'train_test_cali' is not defined

In [27]:
kmeans = KMeans(n_clusters=200, random_state=0).fit(train_test_cali_np)
train_np = train_test_cali.iloc[:90000, :66].to_numpy()
test_np = train_test_cali.iloc[90000:109707, :66].to_numpy()

train_pred = kmeans.predict(train_np)
test_pred = kmeans.predict(test_np)
train['pred_class'] = train_pred
test['pred_class'] = test_pred

In [28]:
quantiles_num = []
train_sum_returns = []
test_sum_returns = []

models = []

for i in range(200):
    
    train_seg = train[train['pred_class']==i]
    test_seg = test[test['pred_class']==i]
    if train_seg.shape[0] == 1:
        continue
    if test_seg.shape[0] < 1:
        continue
    reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
    models.append(reg)
    
    pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

    pred_test = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

    train_res = np.sum(train_seg['TARGET'][pred>0])
    test_res = np.sum(test_seg['TARGET'][pred_test>0])
    
    train_sum_returns.append(train_res)
    test_sum_returns.append(test_res)

In [29]:
print(sum(train_sum_returns))
print(sum(test_sum_returns))

73.8209496065
0.12342044120000008


In [27]:
train_test_np_cons = train_test_np

In [28]:
train_test_np_cons.shape

(109707, 66)

In [ ]:
clf = KMeansConstrained(
     n_clusters=1000,
     size_min=8,
     size_max=125,
     random_state=0
)
clf.fit_predict(train_test_np_cons)

In [ ]:
print(clf.labels_)

In [ ]:
train_test['pred_class'] = clf.labels_

quantiles_num = []
train_sum_returns = []
test_sum_returns = []

models = []

for i in range(1000):
    
    train_seg = train[train['pred_class']==i]
    test_seg = test[test['pred_class']==i]
    quantiles_num.append([train_seg.shape[0], test_seg.shape[0]])
    
    if train_seg.shape[0] == 1:
        continue
    if test_seg.shape[0] < 1:
        continue
    reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seg.iloc[:, :66]), train_seg['TARGET'])
    
    models.append(reg)
    
    train_pred = reg.predict(pd.DataFrame(train_seg.iloc[:, :66]))

    test_pred = reg.predict(pd.DataFrame(test_seg.iloc[:, :66]))

    train_res = np.sum(train_seg['TARGET'][train_pred>0])
    test_res = np.sum(test_seg['TARGET'][test_pred>0])
    
    train_sum_returns.append(train_res)
    test_sum_returns.append(test_res)

print(sum(train_sum_returns))
print(sum(test_sum_returns))